In [17]:
from langchain.document_loaders import WebBaseLoader

# Replace with your target URL
url = "https://github.com/facebookresearch/faiss/blob/main/INSTALL.md"

loader = WebBaseLoader(url)
documents = loader.load()


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [18]:

from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)  # Use 'chunks' instead of overwriting 'documents'

embeddings = OllamaEmbeddings(model="llama2")  # Change model name if needed


C:\Users\Absar Alam\AppData\Local\Temp\ipykernel_9632\2779010916.py:8: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="llama2")  # Change model name if needed


In [ ]:
# Store chunks in Chroma vector database
vectorstore = Chroma.from_documents(chunks, embeddings, persist_directory="./chroma_db")
# vectorstore.persist()

In [ ]:
query = "How to install Faiss on Windows?"
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})
results = retriever.get_relevant_documents(query)

In [ ]:
import os
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings


os.environ['PINECONE_API_KEY'] = 'pcsk_jb8gS_LNy8xi9M59mMaRJi1HtoZG7JKHSx4HxCmGJYW6RhcVoYVMuXAAZ4qQnFhgg7AMe'

index_name = "rag-chatbot"  # Replace with your Pinecone index name

vectorstorePine = PineconeVectorStore(index_name=index_name, embedding=embeddings)

In [38]:

sample_text = "my name is absar alam and i am a software engineer"
vector = embeddings.embed_documents([sample_text])[0]
print("Embedding dimension:", len(vector))

Embedding dimension: 4096


In [ ]:
vectorstorePine.add_documents(chunks)

['29e52c21-851e-4787-9bd2-0dd6d1a4d91a',
 '67ad5494-566a-4fd2-a2c3-29709c6fceb7',
 '21475f4e-b761-4ca4-8bce-dc288497ccd9',
 '8bcb13a2-1174-458f-8e21-117d02ec5b40',
 '28df6c9b-825f-4c62-986e-abcbf3bb08db',
 '00fad64a-38bb-4aea-82dc-aee4baf0abf9',
 'fe0e50d8-fd80-4ea4-a481-72f72e870456',
 'c1892547-8b6d-4ba0-9668-96d74c0d97bc',
 '815c61d9-7540-44a2-bed0-c9671f509414',
 'fbc90e07-0fb9-459a-84c9-8594c55d2e39',
 '03b435b8-56ad-4e59-897f-e9a02e557c35',
 '43a0839d-1fc8-4efa-90b1-47ab0e2ca83e',
 '996bb6c8-48e2-4eaf-9bd1-9be0eba3c289',
 'dedb5f6c-d201-4b73-8777-2023c143209c']

In [ ]:
query = "Installing Faiss via conda"
result =vectorstorePine.similarity_search(query)
result
    


C:\Users\Absar Alam\AppData\Local\Temp\ipykernel_44764\3880201777.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


In [20]:
results

[Document(metadata={'title': 'faiss/INSTALL.md at main · facebookresearch/faiss · GitHub', 'description': 'A library for efficient similarity search and clustering of dense vectors. - faiss/INSTALL.md at main · facebookresearch/faiss', 'source': 'https://github.com/facebookresearch/faiss/blob/main/INSTALL.md', 'language': 'en'}, page_content='Terms\n\n\nPrivacy\n\n\nSecurity\n\n\nStatus\n\n\nDocs\n\n\nContact\n\n\n\n\n       Manage cookies\n    \n\n\n\n\n\n      Do not share my personal information\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    You can’t perform that action at this time.'),
 Document(metadata={'title': 'faiss/INSTALL.md at main · facebookresearch/faiss · GitHub', 'source': 'https://github.com/facebookresearch/faiss/blob/main/INSTALL.md', 'description': 'A library for efficient similarity search and clustering of dense vectors. - faiss/INSTALL.md at main · facebookresearch/faiss', 'language': 'en'}, page_content='Notifications\n You must be signed in to change notification 

In [49]:
from langchain.llms import Ollama
llm = Ollama(model="llama2")  # Change model name if needed

C:\Users\Absar Alam\AppData\Local\Temp\ipykernel_9632\2643046811.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama2")  # Change model name if needed


In [50]:
from langchain_core.prompts import PromptTemplate
prompt =PromptTemplate.from_template (
    """You are an expert assistant. Use the following context to answer the user's question.\n\n"
    <context>
    {context}
    </context>\n\n"
    Question:{input}\n\n
    "Answer:"""
)

# Example usage:
# context = "Faiss can be installed on Windows using conda. Make sure to use the correct Python version."
# user_input = "How do I install Faiss on Windows?"
# print(generate_prompt(context, user_input))

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

doc_chain = create_stuff_documents_chain(llm, prompt=prompt)
db_retriver = vectorstorePine.as_retriever()


In [54]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(db_retriver, doc_chain)

In [55]:
retrieval_chain.invoke({
    "input": "How to install Faiss on Windows?",})

{'input': 'How to install Faiss on Windows?',
 'context': [Document(id='dedb5f6c-d201-4b73-8777-2023c143209c', metadata={'description': 'A library for efficient similarity search and clustering of dense vectors. - faiss/INSTALL.md at main · facebookresearch/faiss', 'language': 'en', 'source': 'https://github.com/facebookresearch/faiss/blob/main/INSTALL.md', 'title': 'faiss/INSTALL.md at main · facebookresearch/faiss · GitHub'}, page_content='Terms\n\n\nPrivacy\n\n\nSecurity\n\n\nStatus\n\n\nDocs\n\n\nContact\n\n\n\n\n       Manage cookies\n    \n\n\n\n\n\n      Do not share my personal information\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    You can’t perform that action at this time.'),
  Document(id='43a0839d-1fc8-4efa-90b1-47ab0e2ca83e', metadata={'description': 'A library for efficient similarity search and clustering of dense vectors. - faiss/INSTALL.md at main · facebookresearch/faiss', 'language': 'en', 'source': 'https://github.com/facebookresearch/faiss/blob/main/INSTALL.md', 't